## Preprocess of AMLsim data for test-for-banks

In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
DATASET = '1K'

In [3]:
path = f'/home/edvin/Desktop/flib/AMLsim/outputs/{DATASET}/tx_log.csv'
df = pd.read_csv(path)
df

,step,type,amount,nameOrig,bankOrig,daysInBankOrig,phoneChangesOrig,oldbalanceOrig,newbalanceOrig,nameDest,bankDest,daysInBankDest,phoneChangesDest,oldbalanceDest,newbalanceDest,isSAR,alertID,modelType
0,0,TRANSFER,110.83,784,bank_d,1,0,68900.32,68789.49,53,bank_a,0,0,62749.07,62859.90,0,-1,11
1,0,TRANSFER,297.86,42,bank_a,1,0,53933.01,53635.14,105,bank_a,0,0,96992.67,97290.54,0,-1,2
2,0,TRANSFER,75.83,394,bank_b,1,1,90738.67,90662.84,985,bank_e,0,0,63516.25,63592.08,0,-1,11
3,0,TRANSFER,87.30,369,bank_b,1,0,93680.79,93593.48,20,bank_a,0,0,72409.24,72496.55,0,-1,1
4,0,TRANSFER,275.52,292,bank_b,1,0,87244.17,86968.65,172,bank_a,0,0,68701.85,68977.37,0,-1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112983,364,TRANSFER,95.44,907,bank_f,1,1,30222.10,30126.65,129,bank_e,2,1,68622.78,68718.23,0,-1,10
112984,364,TRANSFER,165.52,1399,bank_e,0,0,44429.54,44264.01,283,bank_f,1,1,84337.41,84502.94,0,-1,11
112985,364,TRANSFER,181.67,1411,bank_a,0,0,37089.42,36907.74,182,bank_e,0,0,77294.84,77476.51,0,-1,10
112986,364,TRANSFER,215.58,404,bank_b,0,0,10044.55,9828.97,1257,bank_a,3,2,83698.25,83913.83,0,-1,2


In [4]:
df.drop(columns=['type', 'nameOrig', 'bankOrig', 'nameDest', 'bankDest', 'newbalanceOrig', 'oldbalanceOrig', 'newbalanceDest', 'oldbalanceDest', 'alertID', 'modelType'], inplace=True)
df

,step,amount,daysInBankOrig,phoneChangesOrig,daysInBankDest,phoneChangesDest,isSAR
0,0,110.83,1,0,0,0,0
1,0,297.86,1,0,0,0,0
2,0,75.83,1,1,0,0,0
3,0,87.30,1,0,0,0,0
4,0,275.52,1,0,0,0,0
...,...,...,...,...,...,...,...
112983,364,95.44,1,1,2,1,0
112984,364,165.52,0,0,1,1,0
112985,364,181.67,0,0,0,0,0
112986,364,215.58,0,0,3,2,0


In [5]:
df['isSAR'].value_counts()

0    112920
1        68
Name: isSAR, dtype: int64

In [6]:
df_normal = df[df['isSAR'] == 0].sample(n=68)
df_sar = df[df['isSAR'] == 1]
df = pd.concat([df_normal, df_sar])
df

,step,amount,daysInBankOrig,phoneChangesOrig,daysInBankDest,phoneChangesDest,isSAR
1284,4,313.85,0,0,2,1,0
71878,232,180.95,2,1,2,2,0
109471,353,112.43,1,0,0,0,0
49817,160,212.94,1,1,1,1,0
28100,90,65.34,0,0,2,1,0
...,...,...,...,...,...,...,...
108061,349,292.53,2,0,3,1,1
109040,352,297.44,3,0,3,1,1
109398,353,690.68,3,1,2,1,1
110043,355,709.71,4,1,1,1,1


In [7]:
df = df.sample(frac=1.0)
df_train = df.iloc[:int(len(df)*0.8)]
df_test = df.iloc[int(len(df)*0.8):]


In [8]:
X_train = df_train.drop(columns=['isSAR'])
y_train = df_train[['isSAR']]

In [9]:
# df_test_sar = df_test[df_test['isSAR'] == 1]
# df_test_normal = df_test[df_test['isSAR'] == 0].sample(n=len(df_test_sar))
# df_test = pd.concat([df_test_sar, df_test_normal])
X_test = df_test.drop(columns=['isSAR'])
y_test = df_test[['isSAR']]

In [10]:
#X_train = X_train.to_numpy()
#y_train = y_train.to_numpy()
#X_test = X_test.to_numpy()
#y_test = y_test.to_numpy()

In [11]:
os.makedirs(f'./datasets/{DATASET}', exist_ok=True)
#np.savetxt('./datasets/1K/X_train.csv', X_train, delimiter=',', fmt='%10.5f')
#np.savetxt('./datasets/1K/y_train.csv', y_train, delimiter=',', fmt='%10.5f')
#np.savetxt('./datasets/1K/X_test.csv', X_test, delimiter=',', fmt='%10.5f')
#np.savetxt('./datasets/1K/y_test.csv', y_test, delimiter=',', fmt='%10.5f')
X_train.to_csv(f'./datasets/{DATASET}/X_train.csv', index=False)
y_train.to_csv(f'./datasets/{DATASET}/y_train.csv', index=False)
X_test.to_csv(f'./datasets/{DATASET}/X_test.csv', index=False)
y_test.to_csv(f'./datasets/{DATASET}/y_test.csv', index=False)